In [ ]:
!pip install -r requirements.txt

In [1]:
!python --version or python -V

Python 3.12.8


In [1]:
!pip install astropy

In [2]:
import os
from multiprocessing import Pool, cpu_count

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import pandas as pd
from astropy import wcs
from astropy.io import fits
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# def process_file(args) -> None:
#     file_path, destination_path = args
#     os.makedirs(destination_path, exist_ok=True)
#     try:
#         hdu = fits.open(file_path)
#     except Exception as e:
#         print(f"Error opening file {file_path}: {e}")
#         missing_files.append(file_path)
#         return

#     # Extracting information from the header
#     CRPIX1 = hdu[0].header["CRPIX1"]
#     CRPIX2 = hdu[0].header["CRPIX2"]
#     CRVAL1 = hdu[0].header["CRVAL1"]
#     CRVAL2 = hdu[0].header["CRVAL2"]
#     PC1_1 = hdu[0].header["CD1_1"]
#     PC1_2 = hdu[0].header["CD1_2"]
#     PC2_1 = hdu[0].header["CD2_1"]
#     PC2_2 = hdu[0].header["CD2_2"]
#     CTYPE1 = hdu[0].header["CTYPE1"]
#     CTYPE2 = hdu[0].header["CTYPE2"]

#     # Creating WCS object
#     w2 = wcs.WCS(naxis=2)
#     w2.wcs.crpix = [CRPIX1, CRPIX2]
#     w2.wcs.pc = [[PC1_1, PC1_2], [PC2_1, PC2_2]]
#     w2.wcs.crval = [CRVAL1, CRVAL2]
#     w2.wcs.ctype = [CTYPE1, CTYPE2]

#     # Opening a document to write the results
#     # name = re.split('[. /]', file_path)
#     # results = open(name[1] + '_u.txt', 'w')
#     # results.write('X Y RA2000 DEC2000 Value\n')
#     #
#     # # Looping through the data and transforming pixels
#     # for i in range(1, len(hdu[0].data) + 1):
#     #     for j in range(1, len(hdu[0].data[i - 1]) + 1):
#     #         if not np.isnan(hdu[0].data[i - 1][j - 1]):
#     #             # Correcting the indexing
#     #             pixel_coords = np.array([[j, i]])
#     #             radec = w2.pixel_to_world(j+1,i+1)
#     #             #RA_of_the_pixel = radec.ra.hour
#     #
#     #             #DEC_of_the_pixel = radec.dec.degree
#     #             RA_of_the_pixel = 1
#     #
#     #             DEC_of_the_pixel = 1
#     #             #sky_coord = SkyCoord(ra=world_coords[0, 0], dec=world_coords[0, 1], unit=(u.deg, u.deg), frame='fk5')
#     #             #RA_of_the_pixel = sky_coord.ra.to_string(unit=u.hour, sep=':', precision=2, pad=True)
#     #             #DEC_of_the_pixel = sky_coord.dec.to_string(unit=u.degree, sep=':', precision=1, pad=True)
#     #
#     #             results.write('%d %d %s %s %.8f\n' % (j, i, RA_of_the_pixel, DEC_of_the_pixel, hdu[0].data[i - 1][j - 1]))
#     #
#     # # Closing the files
#     # hdu.close()
#     # results.close()
#     data = hdu[0].data
#     y_indices, x_indices = np.indices(data.shape)
#     valid_mask = ~np.isnan(data) & (data != 0)
#     # Filter valid indices
#     x_valid = x_indices[valid_mask]
#     y_valid = y_indices[valid_mask]
#     values_valid = data[valid_mask]
#     # Transform pixel coordinates to world coordinates
#     radec = w2.pixel_to_world(x_valid + 1, y_valid + 1)
#     RA_of_the_pixel, DEC_of_the_pixel = 1, 1  # Placeholder values
#     file_name = file_path.split('/')[-1].split('.')[0]
#     results_filename = f"{file_name}_u.txt"
#     # Write results to file
#     with open(results_filename, 'w') as result_file:
#         result_file.write('X Y RA2000 DEC2000 Value\n')
#         for x, y, value in zip(x_valid, y_valid, values_valid):
#             result_file.write(f'{x + 1} {y + 1} {RA_of_the_pixel} {DEC_of_the_pixel} {value:.8f}\n')
#     hdu.close()
#     #################################
#     data = pd.read_csv(results_filename,sep='\s+',na_values=[""],keep_default_na=False,header=0,encoding='latin1',names=('X','Y','rad','dec','value'))
#     os.remove(results_filename)
#     # data=data.drop(data[(data.value==0)].index)
#     data = data[data.value != 0]

#     plt.scatter(data.X,data.value,marker='.')
#     #plt.show()

#     #skalovanie
#     scaler = MinMaxScaler(feature_range=(-70, 70))
#     y_arr= data.Y.values.reshape(-1, 1)
#     data['Y']=scaler.fit_transform(y_arr)

#     scaler1 = MinMaxScaler(feature_range=(-300, 300))
#     x_arr= data.X.values.reshape(-1, 1)
#     data['X']=scaler1.fit_transform(x_arr)

#     stred_X=(data.X.max()+data.X.min())/2
#     stred_Y=(data.Y.max()+data.Y.min())/2

#     X_max1 = data['value'].idxmax()

#     stred_X1=data['X'].loc[[X_max1]]
#     stred_Y1=data['Y'].loc[[X_max1]]

#     data['X']=data.X-float(stred_X)
#     data['Y']=data.Y-float(stred_Y)

#     plt.scatter(data.X,data.Y,c=data.value,label='_nolegend_', norm=colors.LogNorm())
#     #plt.show()
#     #data.to_csv('tri.txt',sep='\t',index=False,header=False)

#     #rotacia
#     # Fit a linear regression model to find the central line, using a subset of data
#     data_filtered = data[(data['X'] >= -100) & (data['X'] <= 100)]

#     x_range1 = data_filtered['X'].max() - data_filtered['X'].min()
#     y_range1 = data_filtered['Y'].max() - data_filtered['Y'].min()
#     scale_factor = y_range1 / x_range1

#     # Scale the X axis (only for regression)
#     data_scaled = data_filtered.copy()
#     data_scaled['X'] = data_filtered['X'] * scale_factor

#     X = data_scaled['X'].values.reshape(-1, 1)
#     Y = data_scaled['Y'].values
#     reg = LinearRegression().fit(X, Y)
#     slope = reg.coef_[0]
#     intercept = reg.intercept_

#     # Calculate the angle of rotation
#     angle = np.arctan(slope)

#     # Define a function to rotate points
#     def rotate_point(x, y, angle):
#         cos_theta = np.cos(angle)
#         sin_theta = np.sin(angle)
#         x_new = x * cos_theta - y * sin_theta
#         y_new = x * sin_theta + y * cos_theta
#         return x_new, y_new

#     # Rotate the entire galaxy using the angle found
#     data_rot = data.copy()
#     data_rot['X_rot'], data_rot['Y_rot'] = zip(*data.apply(lambda row: rotate_point(row['X'], row['Y'], -angle), axis=1))

#     # Plotting
#     plt.figure(figsize=(8, 6))
#     plt.scatter(data['X'], data['Y'], label='Original Data', color='blue')
#     plt.scatter(data_rot['X_rot'], data_rot['Y_rot'], label='Rotated Data', color='red')
#     plt.xlabel('X')
#     plt.ylabel('Y')
#     plt.title('Central Line of the Galaxy')
#     plt.legend()
#     plt.grid(True)
#     plt.gca().set_aspect('equal', adjustable='box')  # Ensure equal aspect ratio
#     #plt.show()

#     plt.scatter(data_rot.X_rot,data_rot.Y_rot)
#     #plt.show()

#     # Define integer-spaced bins for X_rot and Y_rot
#     x_bins = np.arange(-100, 101, 1)  # X bins from -100 to 100 with step 1
#     y_bins = np.arange(-30, 31, 1)    # Y bins from -30 to 30 with step 1

#     # Assign each point to a bin (ensuring integer values are used for proper binning)
#     data_rot['X_bin'] = np.floor(data_rot['X_rot']).astype(int)
#     data_rot['Y_bin'] = np.floor(data_rot['Y_rot']).astype(int)

#     # Group by the X_bin and Y_bin, and calculate the mean values in each bin
#     binned_data = data_rot.groupby(['X_bin', 'Y_bin']).agg({
#         'X_rot': 'mean',
#         'Y_rot': 'mean',
#         'value': 'mean'
#     }).reset_index()

#     data=data.sort_values(by=['X'])

#     data_rot=data_rot.sort_values(by=['X_rot'])

#     #delime na mensie a vacsie ako 0 v Xku
#     binned_data1=binned_data
#     binned_data=binned_data.drop(binned_data[(binned_data.X_bin>0)].index)
#     binned_data1=binned_data1.drop(binned_data1[(binned_data1.X_bin<0)].index)

#     #skalovat vsetko na rovnake Xmax a Xmin
#     Xmax=300
#     Xmin=-300

#     factor_min=Xmin/binned_data['X_bin'].min()
#     factor_max=Xmax/binned_data1['X_bin'].max()

#     binned_data=binned_data.reset_index(drop=True)
#     binned_data1=binned_data1.reset_index(drop=True)


#     #integral aby sme nasli centrum
#     binned_data=binned_data.sort_values(by=['Y_bin','X_bin'])
#     binned_data=binned_data.reset_index(drop=True)
#     binned_data1=binned_data1.sort_values(by=['Y_bin','X_bin'])
#     binned_data1=binned_data1.reset_index(drop=True)

#     range_x_neg=binned_data.X_bin.drop_duplicates()
#     range_x_neg=range_x_neg.sort_values()
#     range_x_neg=range_x_neg.reset_index(drop=True)

#     range_x_pos=binned_data1.X_bin.drop_duplicates()
#     range_x_pos=range_x_pos.sort_values()
#     range_x_pos=range_x_pos.reset_index(drop=True)

#     def integral(dat,X):
#         dat=dat.drop(dat[(dat.X_bin!=X)].index)
#         dat=dat.reset_index(drop=True)
#         int1,int2=0.0,0.0
#         vysl=list()
#         for i in range(1,len(dat)):
#             dy=dat.Y_bin[i]-dat.Y_bin[i-1]
#             a=dat.value[i]*dat.Y_bin[i]*dy
#             b=dat.value[i]*dy
#             int1+=a
#             int2+=b
#         vysl.append((X,int1,int2))
#         vyslf=pd.DataFrame(vysl,columns=['X','int1','int2'])
#         vyslf['integ']=vyslf.int1/vyslf.int2
#         return vyslf

#     res_neg=pd.DataFrame(columns=['X','int1','int2','integ'])
#     for i in range_x_neg:
#         # res_neg=res_neg._append(integral(binned_data,i),sort=False)
#         res_neg=res_neg.append(integral(binned_data,i),sort=False)


#     res_neg=res_neg.sort_values(by=['X'])
#     res_neg=res_neg.reset_index(drop=True)

#     res_pos=pd.DataFrame(columns=['X','int1','int2','integ'])
#     for i in range_x_pos:
#         # res_pos=res_pos._append(integral(binned_data1,i),sort=False)
#         res_pos=res_pos.append(integral(binned_data1,i),sort=False)

#     res_pos=res_pos.sort_values(by=['X'])
#     res_pos=res_pos.reset_index(drop=True)

#     # Define the bin size
#     bin_size = 30

#     # Find the minimum and maximum X values across both res_neg and res_pos
#     min_x_neg = res_neg['X'].min()
#     max_x_pos = res_pos['X'].max()

#     # Create bin edges ensuring continuity across zero
#     bins = np.arange(min_x_neg, max_x_pos + bin_size, bin_size)

#     # Bin the X values and aggregate
#     res_neg['X_bin'] = pd.cut(res_neg['X'], bins=bins, include_lowest=True, right=False)
#     res_pos['X_bin'] = pd.cut(res_pos['X'], bins=bins, include_lowest=True, right=False)

#     # Aggregate the data by bins
#     res_neg_binned = res_neg.groupby('X_bin').agg({'X': 'mean', 'int1': 'sum', 'int2': 'sum', 'integ': 'mean'}).reset_index(drop=True)
#     res_pos_binned = res_pos.groupby('X_bin').agg({'X': 'mean', 'int1': 'sum', 'int2': 'sum', 'integ': 'mean'}).reset_index(drop=True)

#     # Drop NaN values after binning and reset the index
#     res_neg_binned = res_neg_binned.dropna().reset_index(drop=True)
#     res_pos_binned = res_pos_binned.dropna().reset_index(drop=True)

#     # Combine the binned data
#     res_combined = pd.concat([res_neg_binned, res_pos_binned]).sort_values(by='X').reset_index(drop=True)

#     # Interpolation function for 'integ' values based on the binned data
#     interp_integ = interp1d(res_combined['X'], res_combined['integ'], fill_value="extrapolate", kind='linear')

#     # Add the first and last X values from the original data and interpolate their integ['mean'] values
#     first_x_neg = min_x_neg
#     last_x_pos = max_x_pos

#     # Create a DataFrame for the interpolated values at the edges
#     edge_data = pd.DataFrame({
#         'X': [first_x_neg, last_x_pos],
#         'int1': [np.nan, np.nan],  # We'll only interpolate 'integ'
#         'int2': [np.nan, np.nan],
#         'integ': [interp_integ(first_x_neg), interp_integ(last_x_pos)]
#     })

#     # Append the interpolated values to the combined results
#     res_combined = pd.concat([res_combined, edge_data]).sort_values(by='X').reset_index(drop=True)

#     warp_left = res_combined['integ'].iloc[0]
#     warp_right = res_combined['integ'].iloc[-1]

#     with open(f'{destination_path}/{file_name}_warp.txt', 'w') as warp_file:
#         warp_file.write(f'min: {warp_left}\n')
#         warp_file.write(f'max: {warp_right}\n')

# #     plt.scatter(data_rot.X_rot, data_rot.Y_rot, c=data_rot.value, label='_nolegend_', norm=colors.LogNorm(vmin=10**-3, vmax=1), s=10)
# #     plt.plot(res_combined.X,res_combined.integ,color='red')
# #     plt.colorbar(label='Flux',extend='both')
# #     plt.xlabel('$X$ [Pixels]')
# #     plt.ylabel('$Y_{max}$ [Pixels]')
# # #    plt.ylim(-50,50)
# #     plt.savefig(f'{destination_path}/{file_name}_outline.pdf', format='pdf')
# #     #plt.show()
# #     plt.clf()
# #     plt.close()
#     parts = file_path.split('/')
#     folder_name = parts[-2]  # Extracts 'folder_1'
#     file_base = parts[-1].rsplit('.', 2)[0]  # Removes extensions like .fits.gz2
#     image_path = f"{folder_name}/{file_base}.jpg"
#     galaxy_path = f"{GALAXY_PATH}/{image_path}"
#     mask_path = f"{GALAXY_MASK_PATH}/{image_path}"

#     pdf_path = f'{destination_path}/{file_name}_outline.pdf' 

#     with PdfPages(pdf_path) as pdf:
#         # Page 1: Graph
#         fig, ax = plt.subplots()  # Adjust figure size as needed
#         scatter = ax.scatter(data_rot.X_rot, data_rot.Y_rot, c=data_rot.value, label='_nolegend_', 
#                              norm=colors.LogNorm(vmin=10**-3, vmax=1), s=10)
#         ax.plot(res_combined.X, res_combined.integ, color='red')
#         cbar = plt.colorbar(scatter, ax=ax, label='Flux', extend='both')
#         ax.set_xlabel('$X$ [Pixels]')
#         ax.set_ylabel('$Y_{max}$ [Pixels]')
#         pdf.savefig(fig)  # Save page 1
#         plt.close(fig)  # Close the figure

#         # Page 2: Two Images
#         img1 = mpimg.imread(galaxy_path)
#         img2 = mpimg.imread(mask_path)

#         img1_height, img1_width = img1.shape[:2]
#         img2_height, img2_width = img2.shape[:2]

#         # Maintain aspect ratio while scaling
#         scale_factor = 3  # Adjust this to control final image size
#         new_width1, new_height1 = img1_width * scale_factor, img1_height * scale_factor
#         new_width2, new_height2 = img2_width * scale_factor, img2_height * scale_factor

#         fig, axes = plt.subplots(1, 2, figsize=(new_width1 / 100, new_height1 / 100))  # Scale figsize dynamically

#         for ax in axes:
#             ax.axis('off')

#         axes[0].imshow(img1)
#         axes[1].imshow(img2)

#         pdf.savefig(fig)  # Save page 2
#         plt.close(fig)  # Close the figure

# def get_existing():
#     file_list = [] 
#     for root, dirs, files in os.walk(OUTPUT_DIR): 
#         for file in files:
#             if file.endswith("_outline.pdf"):
#                 file_list.append(file.split("_outline.pdf")[0] + ".fits.bz2")
#     return file_list

# ##### for structure with folders (folder1, folder2 -> data2/uniba )    
# # def collect_files(input_dir):
# #    existing_files = get_existing()
# #    print(f"Found {len(existing_files)} existing files")
# #    tasks = []
# #    for folder in os.listdir(input_dir):
# #        if folder == ".ipynb_checkpoints":
# #            continue
# #        folder_path = os.path.join(input_dir, folder)
# #        for file in os.listdir(folder_path):
# #            #if file.endswith(".fits.bz2") and file not in existing_files:
# #             if file.endswith(".fit.gz") and file not in existing_files:
# #                file_path = os.path.join(folder_path, file)
# #                destination_path = os.path.join(OUTPUT_DIR, folder)
# #                tasks.append((file_path, destination_path))
# #    return tasks

# ##### for simple folder (one folder with fits)
# def collect_files(input_dir):
#    existing_files = get_existing()
#    print(f"Found {len(existing_files)} existing files")
#    tasks = []

#    for file in os.listdir(input_dir):
#            #if file.endswith(".fits.bz2") and file not in existing_files:
#         if file.endswith(".fit.gz") and file not in existing_files:
#            file_path = os.path.join(input_dir, file)
#            tasks.append((file_path, OUTPUT_DIR))
#    return tasks


# ##### for loading fits from list (txt file)
# # def collect_files(txt_file_path):
# # tasks = []

# # # Read file names from the given text file
# # with open(txt_file_path, 'r') as file:
# #     for line in file:
# #         file_name = line.strip()
# #         if file_name:
# #             updated_name = f"{file_name}.fits.bz2"
# #             file_path = f"/home/jovyan/data2/uniba/sdss_extracted_fits/{updated_name}"
# #             tasks.append((file_path, OUTPUT_DIR))

# # return tasks

In [13]:
MASKS_EXTENSION = "png"


def process_file(args) -> None:
    file_path, destination_path = args
    os.makedirs(destination_path, exist_ok=True)

    hdu = fits.open(file_path)

    # Extracting information from the header
    CRPIX1 = hdu[0].header["CRPIX1"]
    CRPIX2 = hdu[0].header["CRPIX2"]
    CRVAL1 = hdu[0].header["CRVAL1"]
    CRVAL2 = hdu[0].header["CRVAL2"]
    PC1_1 = hdu[0].header["CD1_1"]
    PC1_2 = hdu[0].header["CD1_2"]
    PC2_1 = hdu[0].header["CD2_1"]
    PC2_2 = hdu[0].header["CD2_2"]
    CTYPE1 = hdu[0].header["CTYPE1"]
    CTYPE2 = hdu[0].header["CTYPE2"]

    # Creating WCS object
    w2 = wcs.WCS(naxis=2)
    w2.wcs.crpix = [CRPIX1, CRPIX2]
    w2.wcs.pc = [[PC1_1, PC1_2], [PC2_1, PC2_2]]
    w2.wcs.crval = [CRVAL1, CRVAL2]
    w2.wcs.ctype = [CTYPE1, CTYPE2]

    # Opening a document to write the results
    # name = re.split('[. /]', file_path)
    # results = open(name[1] + '_u.txt', 'w')
    # results.write('X Y RA2000 DEC2000 Value\n')
    #
    # # Looping through the data and transforming pixels
    # for i in range(1, len(hdu[0].data) + 1):
    #     for j in range(1, len(hdu[0].data[i - 1]) + 1):
    #         if not np.isnan(hdu[0].data[i - 1][j - 1]):
    #             # Correcting the indexing
    #             pixel_coords = np.array([[j, i]])
    #             radec = w2.pixel_to_world(j+1,i+1)
    #             #RA_of_the_pixel = radec.ra.hour
    #
    #             #DEC_of_the_pixel = radec.dec.degree
    #             RA_of_the_pixel = 1
    #
    #             DEC_of_the_pixel = 1
    #             #sky_coord = SkyCoord(ra=world_coords[0, 0], dec=world_coords[0, 1], unit=(u.deg, u.deg), frame='fk5')
    #             #RA_of_the_pixel = sky_coord.ra.to_string(unit=u.hour, sep=':', precision=2, pad=True)
    #             #DEC_of_the_pixel = sky_coord.dec.to_string(unit=u.degree, sep=':', precision=1, pad=True)
    #
    #             results.write('%d %d %s %s %.8f\n' % (j, i, RA_of_the_pixel, DEC_of_the_pixel, hdu[0].data[i - 1][j - 1]))
    #
    # # Closing the files
    # hdu.close()
    # results.close()
    data = hdu[0].data
    y_indices, x_indices = np.indices(data.shape)
    valid_mask = ~np.isnan(data) & (data != 0)
    # Filter valid indices
    x_valid = x_indices[valid_mask]
    y_valid = y_indices[valid_mask]
    values_valid = data[valid_mask]
    # Transform pixel coordinates to world coordinates
    radec = w2.pixel_to_world(x_valid + 1, y_valid + 1)
    RA_of_the_pixel, DEC_of_the_pixel = 1, 1  # Placeholder values
    file_name = Path(file_path).stem.split('.')[0]
    results_filename = f"{file_name}_u.txt"
    # Write results to file
    with open(results_filename, 'w') as result_file:
        result_file.write('X Y RA2000 DEC2000 Value\n')
        for x, y, value in zip(x_valid, y_valid, values_valid):
            result_file.write(f'{x + 1} {y + 1} {RA_of_the_pixel} {DEC_of_the_pixel} {value:.8f}\n')
    hdu.close()
    #################################
    data = pd.read_csv(results_filename,sep='\s+',na_values=[""],keep_default_na=False,header=0,encoding='latin1',names=('X','Y','rad','dec','value'))
    os.remove(results_filename)
    # data=data.drop(data[(data.value==0)].index)
    data = data[data.value != 0]

    plt.scatter(data.X,data.value,marker='.')
    #plt.show()

    #skalovanie
    scaler = MinMaxScaler(feature_range=(-30, 30))
    y_arr= data.Y.values.reshape(-1, 1)
    data['Y']=scaler.fit_transform(y_arr)

    scaler1 = MinMaxScaler(feature_range=(-50, 50))
    x_arr= data.X.values.reshape(-1, 1)
    data['X']=scaler1.fit_transform(x_arr)

    stred_X=(data.X.max()+data.X.min())/2
    stred_Y=(data.Y.max()+data.Y.min())/2

    X_max1 = data['value'].idxmax()

    stred_X1=data['X'].loc[[X_max1]]
    stred_Y1=data['Y'].loc[[X_max1]]

    data['X']=data.X-float(stred_X)
    data['Y']=data.Y-float(stred_Y)

    plt.scatter(data.X,data.Y,c=data.value,label='_nolegend_', norm=colors.LogNorm())
    #plt.show()
    #data.to_csv('tri.txt',sep='\t',index=False,header=False)

    #rotacia
    # Fit a linear regression model to find the central line, using a subset of data
    data_filtered = data[(data['X'] >= -100) & (data['X'] <= 100)]

    x_range1 = data_filtered['X'].max() - data_filtered['X'].min()
    y_range1 = data_filtered['Y'].max() - data_filtered['Y'].min()
    scale_factor = y_range1 / x_range1

    # Scale the X axis (only for regression)
    data_scaled = data_filtered.copy()
    data_scaled['X'] = data_filtered['X'] * scale_factor

    X = data_scaled['X'].values.reshape(-1, 1)
    Y = data_scaled['Y'].values
    reg = LinearRegression().fit(X, Y)
    slope = reg.coef_[0]
    intercept = reg.intercept_

    # Calculate the angle of rotation
    angle = np.arctan(slope)

    # Define a function to rotate points
    def rotate_point(x, y, angle):
        cos_theta = np.cos(angle)
        sin_theta = np.sin(angle)
        x_new = x * cos_theta - y * sin_theta
        y_new = x * sin_theta + y * cos_theta
        return x_new, y_new

    # Rotate the entire galaxy using the angle found
    data_rot = data.copy()
    data_rot['X_rot'], data_rot['Y_rot'] = zip(*data.apply(lambda row: rotate_point(row['X'], row['Y'], -angle), axis=1))

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.scatter(data['X'], data['Y'], label='Original Data', color='blue')
    plt.scatter(data_rot['X_rot'], data_rot['Y_rot'], label='Rotated Data', color='red')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Central Line of the Galaxy')
    plt.legend()
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')  # Ensure equal aspect ratio
    #plt.show()

    plt.scatter(data_rot.X_rot,data_rot.Y_rot)
    #plt.show()

    # Define integer-spaced bins for X_rot and Y_rot
    x_bins = np.arange(-100, 101, 1)  # X bins from -100 to 100 with step 1
    y_bins = np.arange(-30, 31, 1)    # Y bins from -30 to 30 with step 1

    # Assign each point to a bin (ensuring integer values are used for proper binning)
    data_rot['X_bin'] = np.floor(data_rot['X_rot']).astype(int)
    data_rot['Y_bin'] = np.floor(data_rot['Y_rot']).astype(int)

    # Group by the X_bin and Y_bin, and calculate the mean values in each bin
    binned_data = data_rot.groupby(['X_bin', 'Y_bin']).agg({
        'X_rot': 'mean',
        'Y_rot': 'mean',
        'value': 'mean'
    }).reset_index()

    data=data.sort_values(by=['X'])

    data_rot=data_rot.sort_values(by=['X_rot'])

    #delime na mensie a vacsie ako 0 v Xku
    binned_data1=binned_data
    binned_data=binned_data.drop(binned_data[(binned_data.X_bin>0)].index)
    binned_data1=binned_data1.drop(binned_data1[(binned_data1.X_bin<0)].index)

    #skalovat vsetko na rovnake Xmax a Xmin
    Xmax=300
    Xmin=-300

    factor_min=Xmin/binned_data['X_bin'].min()
    factor_max=Xmax/binned_data1['X_bin'].max()

    binned_data=binned_data.reset_index(drop=True)
    binned_data1=binned_data1.reset_index(drop=True)


    #integral aby sme nasli centrum
    binned_data=binned_data.sort_values(by=['Y_bin','X_bin'])
    binned_data=binned_data.reset_index(drop=True)
    binned_data1=binned_data1.sort_values(by=['Y_bin','X_bin'])
    binned_data1=binned_data1.reset_index(drop=True)

    range_x_neg=binned_data.X_bin.drop_duplicates()
    range_x_neg=range_x_neg.sort_values()
    range_x_neg=range_x_neg.reset_index(drop=True)

    range_x_pos=binned_data1.X_bin.drop_duplicates()
    range_x_pos=range_x_pos.sort_values()
    range_x_pos=range_x_pos.reset_index(drop=True)

    def integral(dat,X):
        dat=dat.drop(dat[(dat.X_bin!=X)].index)
        dat=dat.reset_index(drop=True)
        int1,int2=0.0,0.0
        vysl=list()
        for i in range(1,len(dat)):
            dy=dat.Y_bin[i]-dat.Y_bin[i-1]
            a=dat.value[i]*dat.Y_bin[i]*dy
            b=dat.value[i]*dy
            int1+=a
            int2+=b
        vysl.append((X,int1,int2))
        vyslf=pd.DataFrame(vysl,columns=['X','int1','int2'])
        vyslf['integ']=vyslf.int1/vyslf.int2
        return vyslf

    res_neg=pd.DataFrame(columns=['X','int1','int2','integ'])
    for i in range_x_neg:
        res_neg=res_neg._append(integral(binned_data,i),sort=False)
        #res_neg=res_neg.append(integral(binned_data,i),sort=False)


    res_neg=res_neg.sort_values(by=['X'])
    res_neg=res_neg.reset_index(drop=True)

    res_pos=pd.DataFrame(columns=['X','int1','int2','integ'])
    for i in range_x_pos:
        res_pos=res_pos._append(integral(binned_data1,i),sort=False)
        #res_pos=res_pos.append(integral(binned_data1,i),sort=False)

    res_pos=res_pos.sort_values(by=['X'])
    res_pos=res_pos.reset_index(drop=True)

    # Define the bin size
    bin_size = 5

    # Find the minimum and maximum X values across both res_neg and res_pos
    min_x_neg = res_neg['X'].min()
    max_x_pos = res_pos['X'].max()

    # Create bin edges ensuring continuity across zero
    bins = np.arange(min_x_neg, max_x_pos + bin_size, bin_size)

    # Bin the X values and aggregate
    res_neg['X_bin'] = pd.cut(res_neg['X'], bins=bins, include_lowest=True, right=False)
    res_pos['X_bin'] = pd.cut(res_pos['X'], bins=bins, include_lowest=True, right=False)

    # Aggregate the data by bins
    res_neg_binned = res_neg.groupby('X_bin').agg({'X': 'mean', 'int1': 'sum', 'int2': 'sum', 'integ': 'mean'}).reset_index(drop=True)
    res_pos_binned = res_pos.groupby('X_bin').agg({'X': 'mean', 'int1': 'sum', 'int2': 'sum', 'integ': 'mean'}).reset_index(drop=True)

    # Drop NaN values after binning and reset the index
    res_neg_binned = res_neg_binned.dropna().reset_index(drop=True)
    res_pos_binned = res_pos_binned.dropna().reset_index(drop=True)

    # Combine the binned data
    res_combined = pd.concat([res_neg_binned, res_pos_binned]).sort_values(by='X').reset_index(drop=True)

    # Interpolation function for 'integ' values based on the binned data
    interp_integ = interp1d(res_combined['X'], res_combined['integ'], fill_value="extrapolate", kind='linear')

    # Add the first and last X values from the original data and interpolate their integ['mean'] values
    first_x_neg = min_x_neg
    last_x_pos = max_x_pos

    # Create a DataFrame for the interpolated values at the edges
    edge_data = pd.DataFrame({
        'X': [first_x_neg, last_x_pos],
        'int1': [np.nan, np.nan],  # We'll only interpolate 'integ'
        'int2': [np.nan, np.nan],
        'integ': [interp_integ(first_x_neg), interp_integ(last_x_pos)]
    })

    # Append the interpolated values to the combined results
    res_combined = pd.concat([res_combined, edge_data]).sort_values(by='X').reset_index(drop=True)

    warp_left = res_combined['integ'].iloc[0]
    warp_right = res_combined['integ'].iloc[-1]

    #print('min:',warp_left)
    #print('max:',warp_right)
    with open(f'{destination_path}/{file_name}_warp.txt', 'w') as warp_file:
        warp_file.write(f'min: {warp_left}\n')
        warp_file.write(f'max: {warp_right}\n')

    # plt.scatter(data_rot.X_rot, data_rot.Y_rot, c=data_rot.value, label='_nolegend_', norm=colors.LogNorm(vmin=10**-3, vmax=1), s=10)
    # plt.plot(res_combined.X,res_combined.integ,color='red')
    # plt.colorbar(label='Flux',extend='both')
    # plt.xlabel('$X$ [Pixels]')
    # plt.ylabel('$Y_{max}$ [Pixels]')
    # #plt.ylim(-50,50)
    # plt.savefig(f'{destination_path}/{file_name}_outline.pdf', format='pdf')
    # #plt.show()

    #parts = file_path.split('/')
    #folder_name = parts[-2]  # Extracts 'folder_1'
    file_base = Path(file_path).stem.rsplit('.', 2)[0]  # Removes extensions like .fits.gz2
    #image_path = f"{file_base}.png"
    #galaxy_path = f"{GALAXY_PATH}/{image_path}"
    galaxy_path = f"{GALAXY_PATH}/{file_base}.jpg"
    mask_path = f"{GALAXY_MASK_PATH}/{file_base}.{MASKS_EXTENSION}"

    pdf_path = f'{destination_path}/{file_name}_outline_bin5.pdf'

    with PdfPages(pdf_path) as pdf:
        # Page 1: Graph
        fig, ax = plt.subplots()  # Adjust figure size as needed
        scatter = ax.scatter(data_rot.X_rot, data_rot.Y_rot, c=data_rot.value, label='_nolegend_',
                             norm=colors.LogNorm(), s=10)
        ax.plot(res_combined.X, res_combined.integ, color='red')
        cbar = plt.colorbar(scatter, ax=ax, label='Flux', extend='both')
        ax.set_xlabel('$X$ [Pixels]')
        ax.set_ylabel('$Y_{max}$ [Pixels]')
        pdf.savefig(fig)  # Save page 1
        plt.close(fig)  # Close the figure

        # Page 2: Two Images
        img1 = mpimg.imread(galaxy_path)
        img2 = mpimg.imread(mask_path)

        img1_height, img1_width = img1.shape[:2]
        img2_height, img2_width = img2.shape[:2]

        # Maintain aspect ratio while scaling
        scale_factor = 3  # Adjust this to control final image size
        new_width1, new_height1 = img1_width * scale_factor, img1_height * scale_factor
        new_width2, new_height2 = img2_width * scale_factor, img2_height * scale_factor

        fig, axes = plt.subplots(1, 2, figsize=(new_width1 / 100, new_height1 / 100))  # Scale figsize dynamically

        for ax in axes:
            ax.axis('off')

        axes[0].imshow(img1)
        axes[1].imshow(img2)

        pdf.savefig(fig)  # Save page 2
        plt.close(fig)  # Close the figure

def collect_files(input_dir):
    tasks = []
    # for folder in os.listdir(input_dir):
    #     if folder == ".ipynb_checkpoints":
    #         continue
    #     folder_path = os.path.join(input_dir, folder)
    for file in os.listdir(input_dir):
        if file.endswith(".fits"):
            file_path = os.path.join(input_dir, file)
            # destination_path = os.path.join(OUTPUT_DIR, folder)
            destination_path = os.path.join(OUTPUT_DIR)
            tasks.append((file_path, destination_path))
    return tasks

In [14]:
#INPUT_DIR = '/home/jovyan/data2/uniba/sdss_extracted_fits'
#INPUT_DIR = '/home/jovyan/data2/uniba/file_paths.txt'
#OUTPUT_DIR = '/home/jovyan/data2/uniba/sdss_top_galaxies_warp_calculations'
#GALAXY_PATH = '/home/jovyan/data2/uniba/sdss_cropped_galaxies'
#GALAXY_MASK_PATH = '/home/jovyan/data2/uniba/sdss_cropped_masks'

INPUT_DIR = '/home/jovyan/data/lightning/JulianaGazdova/data/new_data_subset/rucne_masky/cut_fits_masked'
OUTPUT_DIR = '/home/jovyan/data/lightning/JulianaGazdova/data/new_data_subset/rucne_masky/warp_calculations'
GALAXY_PATH = '/home/jovyan/data/lightning/JulianaGazdova/data/new_data_subset/galaxies'
GALAXY_MASK_PATH = '/home/jovyan/data/lightning/JulianaGazdova/data/new_data_subset/rucne_masky/masks_scss'
missing_files = []

tasks = collect_files(INPUT_DIR)
print(f"Found {len(tasks)} files to process.")

# Use multiprocessing to process files
with Pool(processes=cpu_count()) as pool:
    pool.map(process_file, tasks)

Found 6 files to process.
